In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import torch
print(torch.cuda.is_available())


True


In [10]:
import torch
from torch import cuda
from spandrel import ImageModelDescriptor, ModelLoader
from PIL import Image
import torchvision.transforms as transforms
from pathlib import Path
import gc  # Import garbage collector

# Constants for path and normalization
MODEL_FILENAME = "8x_NMKD-Typescale_175k.pth"
INPUT_IMAGE_FILENAME = "Fine.jpeg"
OUTPUT_IMAGE_FILENAME = "upscaled_Fine_spandrel.jpeg"
NORMALIZE_MEAN = [0.5, 0.5, 0.5]
NORMALIZE_STD = [0.5, 0.5, 0.5]
PATCH_SIZE = (256, 256)  # Size of each patch

# Define paths using pathlib for better path handling
current_dir = Path.cwd()
model_path = current_dir / MODEL_FILENAME
input_image_path = current_dir / INPUT_IMAGE_FILENAME
output_image_path = current_dir / OUTPUT_IMAGE_FILENAME

# Load model function
def load_model(model_path: Path) -> ImageModelDescriptor:
    model = ModelLoader().load_from_file(str(model_path))
    assert isinstance(model, ImageModelDescriptor), "The loaded model must be an ImageModelDescriptor."
    if cuda.is_available():
        model = model.cuda()
        model = model.half()  # Convert model to half precision
    model.eval()
    return model

# Upscale image function with memory management
def upscale_image(model, input_image: torch.Tensor) -> torch.Tensor:
    H, W = input_image.shape[2], input_image.shape[3]
    scale_factor = 8  # Replace with actual scale factor from your model if different
    full_output = torch.zeros(3, H*scale_factor, W*scale_factor).half() if cuda.is_available() else torch.zeros(3, H*scale_factor, W*scale_factor)
    
    for i in range(0, H, PATCH_SIZE[0]):
        for j in range(0, W, PATCH_SIZE[1]):
            input_patch = input_image[:, :, i:i+PATCH_SIZE[0], j:j+PATCH_SIZE[1]]
            ih, iw = i*scale_factor, j*scale_factor
            if cuda.is_available():
                input_patch = input_patch.cuda().half()
                with torch.no_grad():
                    output_patch = model(input_patch)
                    full_output[:, ih:ih+output_patch.shape[2], iw:iw+output_patch.shape[3]] = output_patch
                torch.cuda.empty_cache()
            else:
                with torch.no_grad():
                    output_patch = model(input_patch)
                    full_output[:, ih:ih+output_patch.shape[2], iw:iw+output_patch.shape[3]] = output_patch
                   
    if cuda.is_available():
        full_output = full_output.cpu()  # Move the full tensor to CPU
        
    full_output = full_output.float() # Convert back to full precision if it was in half-precision
    
    return full_output

# Save upscaled image function
def save_image(tensor: torch.Tensor, file_path: Path):
    output_image = transforms.ToPILImage()(tensor)
    output_image.save(file_path)
    print("Upscaled image saved successfully!")

# Process the image
def process_image():
    gc.collect()
    if cuda.is_available():
        cuda.empty_cache()

    model = load_model(model_path)
    input_image = Image.open(input_image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=NORMALIZE_MEAN, std=NORMALIZE_STD)
    ])
    input_tensor = transform(input_image).unsqueeze(0)
    output_image = upscale_image(model, input_tensor)
    
    # Save the entire upscaled image
    save_image(output_image.squeeze(0), output_image_path)

if __name__ == "__main__":
    process_image()


Upscaled image saved successfully!


In [7]:
import torch
from torch import cuda
from spandrel import ImageModelDescriptor, ModelLoader
from PIL import Image
import torchvision.transforms as transforms
from pathlib import Path
import gc  # Import garbage collector

# Constants for path and normalization
MODEL_FILENAME = "8x_NMKD-Typescale_175k.pth"
INPUT_IMAGE_FILENAME = "Fine.jpeg"
OUTPUT_IMAGE_FILENAME = "upscaled_Fine_spandrel.jpeg"
NORMALIZE_MEAN = [0.5, 0.5, 0.5]
NORMALIZE_STD = [0.5, 0.5, 0.5]
PATCH_SIZE = (256, 256)  # Size of each patch

# Define paths using pathlib for better path handling
current_dir = Path.cwd()
model_path = current_dir / MODEL_FILENAME
input_image_path = current_dir / INPUT_IMAGE_FILENAME
output_image_path = current_dir / OUTPUT_IMAGE_FILENAME

# Load model function
def load_model(model_path: Path) -> ImageModelDescriptor:
    model = ModelLoader().load_from_file(str(model_path))
    assert isinstance(model, ImageModelDescriptor), "The loaded model must be an ImageModelDescriptor."
    if cuda.is_available():
        model = model.cuda()
        model = model.half()  # Convert model to half precision
    model.eval()
    return model

# Upscale image function with memory management
def upscale_image(model, input_image: torch.Tensor) -> torch.Tensor:
    try:
        if cuda.is_available():
            input_image = input_image.cuda()
            input_image = input_image.half()  # Convert input tensor to half precision
            
        with torch.no_grad():
            output_patches = []
            for i in range(0, input_image.shape[2], PATCH_SIZE[0]):
                for j in range(0, input_image.shape[3], PATCH_SIZE[1]):
                    input_patch = input_image[:, :, i:i+PATCH_SIZE[0], j:j+PATCH_SIZE[1]]
                    output_patch = model(input_patch)
                    output_patches.append(output_patch)
        
        output_image = torch.cat(output_patches, dim=3)  # Concatenate patches along width
        output_image = output_image.float()  # Convert back to full precision if needed
        return output_image.cpu()  # Move tensor back to CPU
    
    except cuda.CudaError as e:
        print(f"An error occurred during CUDA processing: {e}")
        raise
    except Exception as e:
        print(f"An error occurred while upscaling the image: {e}")
        raise
    finally:
        if cuda.is_available():
            torch.cuda.empty_cache()  # Free up unused memory
        gc.collect()  # Run garbage collection

# Save upscaled image function
def save_image(tensor: torch.Tensor, file_path: Path):
    try:
        output_image = transforms.ToPILImage()(tensor)
        output_image.save(file_path)
        print("Upscaled image saved successfully!")
    except Exception as e:
        print(f"An error occurred while saving the image: {e}")
        raise

# Process the image
def process_image():
    # Perform garbage collection and empty the cache before processing
    gc.collect()
    if cuda.is_available():
        cuda.empty_cache()

    model = load_model(model_path)
    input_image = Image.open(input_image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=NORMALIZE_MEAN, std=NORMALIZE_STD)
    ])
    input_tensor = transform(input_image).unsqueeze(0)
    
    # Upscale image
    output_image = upscale_image(model, input_tensor)
    
    # Save the image in smaller patches
    patch_size = 256  # Choose an appropriate patch size
    for i in range(0, output_image.shape[3], patch_size):
        patch = output_image[:, :, :, i:i+patch_size]
        output_patch_path = output_image_path.with_suffix(f".patch_{i}.jpeg")
        try:
            save_image(patch.squeeze(0), output_patch_path)
        except Exception as e:
            print(f"An error occurred while saving patch {i}: {e}")
            raise

if __name__ == "__main__":
    process_image()

Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved successfully!
Upscaled image saved

In [5]:
from spandrel import ImageModelDescriptor, ModelLoader
import torch
from PIL import Image
import torchvision.transforms as transforms
import os

# Load the model from the same directory
model_path = os.path.join(os.getcwd(), "8x_NMKD-Typescale_175k.pth")

# Load the model
model = ModelLoader().load_from_file(model_path)

# Make sure it's an image-to-image model
assert isinstance(model, ImageModelDescriptor)

# Send the model to the GPU and put it in inference mode
model.cuda().eval()

# Define a function to upscale the image
def upscale_image(image_path: str) -> torch.Tensor:
    # Load and preprocess the input image
    input_image = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Assuming the model requires normalization
    ])
    input_image = transform(input_image).unsqueeze(0).cuda()

    # Upscale the image using the model
    with torch.no_grad():
        output_image = model(input_image)

    return output_image.squeeze(0).cpu()

# Process the image
input_image_path = os.path.join(os.getcwd(), "Fine.jpeg")

output_image = upscale_image(input_image_path)

# Save the upscaled image
output_image = transforms.ToPILImage()(output_image)
output_image.save(os.path.join(os.getcwd(), "upscaled_Fine_spandrel.jpeg"))

print("Upscaled image saved successfully!")
